In [3]:
%cd ../..

/media/atem/Data/HSE_videos/4_DLA/hw_2_SeppechSep/git_speech_separation


In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# DTTNet Plus

In [1]:
!pwd

/media/atem/Data/HSE_videos/4_DLA/hw_2_SeppechSep/git_speech_separation/src/notebooks


## Test Dataset

In [4]:
from omegaconf import OmegaConf
from hydra.utils import instantiate
from hydra import initialize, compose

with initialize(config_path="../configs", version_base=None):
    cfg = compose(config_name="baseline.yaml")
    dataset = instantiate(cfg.datasets.train, reindex=False, video_dir="/home/atem/HSE_videos/4_DLA/hw_2_SeppechSep/dla_dataset/mouths")

[ApplyImpulseResponse] No local RIRS found, downloading...
[ApplyImpulseResponse] Downloading from https://www.openslr.org/resources/28/rirs_noises.zip ...


KeyboardInterrupt: 

In [23]:
from src.datasets.collate import collate_fn

In [24]:
import random
k = 10
indices = random.sample(range(len(dataset)), k)
examples = [dataset[idx] for idx in indices]

In [25]:
examples[0]["spectrogram"].shape

torch.Size([1, 513, 126])

In [26]:
batch = collate_fn(examples)

In [27]:
batch["video"].shape

torch.Size([10, 2, 50, 88, 88])

In [28]:
video_batch = batch["video"]

In [29]:
video_batch.shape

torch.Size([10, 2, 50, 88, 88])

## Video Encoder

In [30]:
from src.model.DTTNetPlus.modules.video_encoder import VideoEncoderModule
video_encoder = VideoEncoderModule("/media/atem/Data/HSE_videos/4_DLA/hw_2_SeppechSep/git_speech_separation/src/model/lipreading/lrw_snv1x_dsmstcn3x.pth")

In [31]:
video_encoder_out = video_encoder(video_batch)

In [32]:
video_encoder_out.shape

torch.Size([10, 50, 2048])

## AudioEncoder

In [33]:
fc_dim=(1024 // 2 + 1)
n_layers=3

In [34]:
from src.model.DTTNetPlus.modules.encoder import Encoder
encoder = Encoder(fc_dim=(1024 // 2 + 1))

In [35]:
encoder_out_x, encoder_out_skip = encoder(batch["spectrogram"], batch["phase"])

In [36]:
encoder_out_x.shape

torch.Size([10, 256, 65, 18])

In [37]:
from src.model.DTTNet.blocks.block_tfc_tdf import TFC_TDF_Block
tfc_tdf = TFC_TDF_Block(channels=256, 
                        fc_dim=((fc_dim + 2 ** n_layers) // 2 ** n_layers)
                        )

In [38]:
out_tfc_tdf = tfc_tdf(encoder_out_x)

In [39]:
out_tfc_tdf.shape

torch.Size([10, 256, 65, 18])

## FUSION BLOCK

In [40]:
import torch.nn.functional as Fun

In [41]:
encoder_out_x.shape, video_encoder_out.shape

(torch.Size([10, 256, 65, 18]), torch.Size([10, 50, 2048]))

In [45]:
65 * 256

16640

In [42]:
B, C, F, T = encoder_out_x.shape
video_interpolated = Fun.interpolate(video_encoder_out.transpose(-1,-2), size=T, mode="linear").transpose(-1,-2)

In [43]:
video_interpolated.shape

torch.Size([10, 18, 2048])

In [47]:
audio_data = encoder_out_x

In [48]:
audio_data.permute(0, 3, 1, 2).flatten(-2, -1).shape

torch.Size([10, 18, 16640])

In [65]:
import torch
from torch import nn
from torch.nn import functional as F


class FusionBlock(nn.Module):
    def __init__(self, video_channels, audio_channels, audio_freq):
        super().__init__()

        self.w_gamma_channels = nn.Linear(video_channels, audio_channels)
        self.w_beta_channels = nn.Linear(video_channels, audio_channels)

        self.w_gamma_freq = nn.Linear(video_channels, audio_freq)
        self.w_beta_freq = nn.Linear(video_channels, audio_freq)

    def forward(self, audio_data: torch.Tensor, video_data: torch.Tensor):
        B, C, F, T = audio_data.shape
        video_interpolated = Fun.interpolate(video_encoder_out.transpose(-1,-2), size=T, mode="linear").transpose(-1,-2)

        gamma_c = self.w_gamma_channels(video_interpolated)
        beta_c = self.w_beta_channels(video_interpolated)

        gamma_f = self.w_gamma_freq(video_interpolated)
        beta_f = self.w_beta_freq(video_interpolated)

        gamma_c = gamma_c.permute(0, 2, 1).unsqueeze(2)   # [B, C, 1, T]
        gamma_f = gamma_f.permute(0, 2, 1).unsqueeze(1)   # [B, 1, F, T]
        beta_c  = beta_c.permute(0, 2, 1).unsqueeze(2)    # [B, C, 1, T]
        beta_f  = beta_f.permute(0, 2, 1).unsqueeze(1)    # [B, 1, F, T]

        gamma = gamma_c + gamma_f        # [B, C, F, T] 
        beta  = beta_c + beta_f          # [B, C, F, T]

        fused_signal = (1 + gamma) * audio_data + beta

        return fused_signal




In [74]:
from src.model.DTTNetPlus.model_dttnet import DTTNetPlusModel
model = DTTNetPlusModel()

In [60]:
encoder_out_x.shape, video_encoder_out.shape

(torch.Size([10, 256, 65, 18]), torch.Size([10, 50, 2048]))

In [67]:
fusion = FusionBlock(video_channels=2048, audio_channels=256, audio_freq=65)

In [68]:
fused_data = fusion(encoder_out_x, video_encoder_out)

In [69]:
fused_data.shape

torch.Size([10, 256, 65, 18])

In [75]:
model(**batch)

{'preds': [{'audio': tensor([[-6.4221e-03,  2.0777e-04, -7.5337e-03,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [ 1.2979e+00,  1.5339e+00,  1.6751e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [-2.4915e-02, -1.2307e-02, -1.7316e-02,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           ...,
           [-4.4063e-03,  3.3000e-03,  4.1825e-04,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [ 6.2423e-01,  5.8092e-01,  5.3005e-01,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [-1.7233e-02, -1.0318e-02, -1.4414e-02,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00]], grad_fn=<ConstantPadNdBackward0>),
   'spectrogram': tensor([[[-1.2674e-05,  3.9557e-01,  4.3761e-01,  ...,  0.0000e+00,
              0.0000e+00,  0.0000e+00],
            [-4.7159e-01, -2.9523e-01,  8.1691e-02,  ...,  0.0000e+00,
              0.0000e+00,  0.0000e+00],
            [ 3.6916e-01,  2.1221e-01, 